# Gradient Boosting Machine (GBM)

부스팅 알고리즘: 여러 개의 weak learner를 순차적으로 학습-예측하며 잘못 예측한 데이터에 가중치를 부여하면서 오류를 개선해 나가는 학습 방식

- AdaBoost (Adaptive Boosting)
- Gradient Boost Machine: 가중치 업데이트에 Gradient Descent를 이용

## GBM을 이용한 사용자 행동 인식 데이터에 대한 예측 분류

수집된 피처로 어떤 동작인지 예측

In [11]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

pandas 신규 버전: 중복된 Feature 명으로 인해 Duplicate name 에러 발생.

중복 feature명에 대해서 원본 feature 명에 '_1(또는2)'를 추가로 부여하는 함수인 get_new_feature_name_df() 생성

In [6]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

new_feature_name_df = get_new_feature_name_df(feature_name_df)

In [8]:
def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [12]:
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

GBM 정확도: 0.9382
GBM 수행 시간: 575.7 초 


In [13]:
print(f'GBM 정확도: {gb_accuracy}')
print(f'GBM 수행시간: {time.time() - start_time} 초')

GBM 정확도: 0.9382422802850356
GBM 수행시간: 593.6192271709442 초


- 일반적으로 GBM이 랜덤포레스트보다 예측 성능 뛰어난 경우가 많음
- 하지만 수행시간 ↑, 하이퍼 파라미터 튜닝 더 까다로움
    - weak learner의 순차적 예측 오류 보정을 통한 학습 → 병렬 처리 불가.. 시간 매우 오래 걸림

## 하이퍼 파라미터
- loss: 기본값 deviance
- learning_rate: 기본값 0.1, n_estimators와 상호보완적으로 사용
- n_estimators: weak learner의 개수. 기본값 100
- subsample: weak learner가 학습에 사용하는 데이터의 샘플링 비율. 기본값 1(전체 데이터 사용). 과적합 → 1보다 작은 값으로 설정

In [ ]:
## 하이퍼 파라미터 최적화

## 너무 오래 걸려서 코드 돌려보진 않음

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 500],
    'learning_rate': [0.05, 0.1]    
}
grid_cv = GridSearchCV(gb_clf, param_grid = params, cv = 2, verbose = 1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

**sklearn.model_selection.GridSearchCV()**

**verbose**: integer

Controls the verbosity: the higher, the more messages.

log 출력의 level을 조정 (숫자가 클 수록 많은 log 출력)

In [ ]:
# test set 예측 정확도 

gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

> GBM은 과적합에도 강한 뛰어난 예측 성능을 가지지만 수행 시간이 오래 걸린다.

> GBM 기반 ML 패키지 XGBoost, LightGBM 등